# Lesson 6: Shipping as a web API

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { 
  loadAndSplitChunks, 
  initializeVectorstoreWithDocuments 
} from "./lib/helpers.ts";

const splitDocs = await loadAndSplitChunks({
  chunkSize: 1536,
  chunkOverlap: 128,
});

const vectorstore = await initializeVectorstoreWithDocuments({
  documents: splitDocs,
});

const retriever = vectorstore.asRetriever();

In [3]:
import { 
  createDocumentRetrievalChain, 
  createRephraseQuestionChain 
} from "./lib/helpers.ts";

const documentRetrievalChain = createDocumentRetrievalChain();
const rephraseQuestionChain = createRephraseQuestionChain();

In [4]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const ANSWER_CHAIN_SYSTEM_TEMPLATE = `You are an experienced researcher,
expert at interpreting and answering questions based on provided sources.
Using the below provided context and chat history, 
answer the user's question to the best of your ability
using only the resources provided. Be verbose!

<context>
{context}
</context>`;

const answerGenerationChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", ANSWER_CHAIN_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  [
    "human", 
    `Now, answer this question using the previous context and chat history:
  
    {standalone_question}`
  ]
]);

In [5]:
import { 
  RunnablePassthrough, 
  RunnableSequence 
} from "@langchain/core/runnables";
import { ChatOpenAI } from "@langchain/openai";

const conversationalRetrievalChain = RunnableSequence.from([
  RunnablePassthrough.assign({
    standalone_question: rephraseQuestionChain,
  }),
  RunnablePassthrough.assign({
    context: documentRetrievalChain,
  }),
  answerGenerationChainPrompt,
  new ChatOpenAI({ modelName: "gpt-3.5-turbo-1106" }),
]);

In [6]:
import { HttpResponseOutputParser } from "langchain/output_parsers";

// "text/event-stream" is also supported
const httpResponseOutputParser = new HttpResponseOutputParser({
  contentType: "text/plain"
});

In [7]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables"; 
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const messageHistory = new ChatMessageHistory();

const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: (_sessionId) => messageHistory,
  historyMessagesKey: "history",
  inputMessagesKey: "question",
}).pipe(httpResponseOutputParser);

Additionally, we'll want to bear in mind that users should not share chat histories, and we should create a new history object per session:

In [8]:
const messageHistories = {};

const getMessageHistoryForSession = (sessionId) => {
    if (messageHistories[sessionId] !== undefined) {
        return messageHistories[sessionId];
    } 
    const newChatSessionHistory = new ChatMessageHistory();
    messageHistories[sessionId] = newChatSessionHistory;
    return newChatSessionHistory;
};

We'll recreate our final chain with this new method:

In [9]:
const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: getMessageHistoryForSession,
  inputMessagesKey: "question",
  historyMessagesKey: "history",
}).pipe(httpResponseOutputParser);

In [10]:
const port = 8087;

In [11]:
const handler = async (request: Request): Response => {
  const body = await request.json();
  const stream = await finalRetrievalChain.stream({
    question: body.question
  }, { configurable: { sessionId: body.session_id } });

  return new Response(stream, { 
    status: 200,
    headers: {
      "Content-Type": "text/plain"
    },
  });
};

In [12]:
Deno.serve({ port }, handler);

Listening on http://localhost:8087/


{
  finished: Promise { <pending> },
  shutdown: [AsyncFunction: shutdown],
  ref: [Function: ref],
  unref: [Function: unref],
  [Symbol(Symbol.asyncDispose)]: [Function: [Symbol.asyncDispose]]
}

In [13]:
const decoder = new TextDecoder();

// readChunks() reads from the provided reader and yields the results into an async iterable
function readChunks(reader) {
  return {
    async* [Symbol.asyncIterator]() {
      let readResult = await reader.read();
      while (!readResult.done) {
        yield decoder.decode(readResult.value);
        readResult = await reader.read();
      }
    },
  };
}

const sleep = async () => {
  return new Promise((resolve) => setTimeout(resolve, 500));
}

In [14]:
const response = await fetch(`http://localhost:${port}`, {
    method: "POST",
    headers: {
        "content-type": "application/json",
    },
    body: JSON.stringify({
        question: "What are the prerequisites for this course?",
        session_id: "1", // Should randomly generate/assign
    })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: The prerequisit
CHUNK: es for t
CHUNK: he course w
CHUNK: ere outlined
CHUNK:  by the instruc
CHUNK: tor in the prov
CHUNK: ided con
CHUNK: text and chat his
CHUNK: tory. The 
CHUNK: instructor emphasi
CHUNK: zed the need
CHUNK:  for familiar
CHUNK: ity with basic
CHUNK:  probability and 
CHUNK: statistics,
CHUNK:  basic linear alge
CHUNK: bra, and p
CHUNK: rogramming k
CHUNK: nowledge
CHUNK: . Specifically,
CHUNK:  the prerequisites include
CHUNK: :

1. Basic 
CHUNK: Probability and Statistics:
 
CHUNK:   - Familiarity with rand
CHUNK: om variables
CHUNK: , expectatio
CHUNK: n, variance
CHUNK: , and probability conc
CHUNK: epts.
   - The 
CHUNK: assumption th
CHUNK: at stude
CHUNK: nts have ta
CHUNK: ken an unde
CHUNK: rgraduate 
CHUNK: statistics class, 
CHUNK: such as
CHUNK:  Stat 116 at Sta
CHUNK: nford, or
CHUNK:  an equivalent
CHUNK:  course.

2. Basic Linear A
CHUNK: lgebra:
   - Familiarity with concepts re
CHUNK: lated to mat
CHUNK: rices and ve
CHUNK: ctors, including 

In [15]:
const response = await fetch(`http://localhost:${port}`, {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "Can you list them in bullet point format?",
    session_id: "1", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: Certainl
CHUNK: y! Based on th
CHUNK: e provided context a
CHUNK: nd chat his
CHUNK: tory, the prereq
CHUNK: uisites for the 
CHUNK: course can be sum
CHUNK: marized in
CHUNK:  bullet point
CHUNK:  format as fol
CHUNK: lows:

-
CHUNK:  Basic Probability
CHUNK:  and Statistics:
  -
CHUNK:  Familiar
CHUNK: ity with rando
CHUNK: m variables, expectati
CHUNK: on, variance,
CHUNK:  and probability conce
CHUNK: pts.
  - Complet
CHUNK: ion of an undergrad
CHUNK: uate stati
CHUNK: stics class l
CHUNK: ike Stat 11
CHUNK: 6 at Stanford
CHUNK:  or an equival
CHUNK: ent course.
CHUNK: 

- Basic Linear A
CHUNK: lgebra:
  - Familiarity with concepts
CHUNK:  related 
CHUNK: to matrices and 
CHUNK: vectors,
CHUNK:  includin
CHUNK: g matrix multi
CHUNK: plication, matrix i
CHUNK: nverses, and eigen
CHUNK: vectors.
  - Backgr
CHUNK: ound from undergraduate 
CHUNK: linear algebra
CHUNK:  courses like Math 51
CHUNK: , 103, Math 113,
CHUNK:  or CS205 at Stanford is
CHUNK:  sufficien
CHUNK: t.

- Pro

In [16]:
const response = await fetch(`http://localhost:${port}`, {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "What did I just ask you?",
    session_id: "2", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: Based on 
CHUNK: the provi
CHUNK: ded context 
CHUNK: and chat
CHUNK:  history, 
CHUNK: you did not ask a 
CHUNK: specific 
CHUNK: question. 
CHUNK: Instead,
CHUNK:  the instruct
CHUNK: or is inquir
CHUNK: ing about
CHUNK:  the backgrou
CHUNK: nd of the stu
CHUNK: dents in the class,
CHUNK:  asking which disciplines
CHUNK:  they are f
CHUNK: rom. The
CHUNK:  instructor goes on to a
CHUNK: cknowledge the
CHUNK:  diversity of the students
CHUNK: , indicating t
CHUNK: hat it mak
CHUNK: es the class interesting to teach and be in. Therefore, based on the context provided, I can infer that you did not ask a question, but rather participated in a class discussion where the instructor was gathering information about the students' academic backgrounds.
